### Домашнее задание №2 Бейзлайны и детерминированные алгоритмы item-item

In [1]:
# !pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('C:/Users/Larin-GV/RecSys/retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    rnd = random.rand(n)
    itm,wgt = np.array(items_weights['item_id']),np.array(items_weights['weight']).cumsum()
    indx = wgt.searchsorted(rnd)
    recs = itm[indx] 
    
    return recs.tolist()

In [6]:
%%time
from numpy import random
# your_code

df = data.groupby('item_id')['quantity'].sum().reset_index()
df.columns = ['item_id','weight']
df['weight'] = df['weight'].apply(lambda x: np.log(x+1))
sumw = df['weight'].sum()
df['weight'] = df['weight'].apply(lambda x: x/sumw)

recs = weighted_random_recommendation(df)
print(recs)

[934045, 1078317, 6552728, 12731583, 826467]
Wall time: 1.07 s


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
result = pd.read_csv('C:/Users/Larin-GV/RecSys/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [8]:
def fromStr(stri):
    stri=stri[1:-1]    
    stri = stri.replace('\n','')
    stri = stri.replace(',','')
    lst = stri.split()
    ids = [int(en) for en in lst]
    return ids

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    if len(recommended_list)==0: return 1
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [10]:
%%time
# your_code

prcs=np.zeros(6)
for i in range(result.shape[0]):
    for j in range(2,8):
        prcs[j-2] += precision_at_k(fromStr(result.iloc[i,j]), fromStr(result.iloc[i,1]))

print(prcs/result.shape[0])

[0.00058766 0.15523996 0.03359452 0.03525955 0.03614104 0.17998694]
Wall time: 2.12 s


### Вывод:  лучшими предсказателями  стали: 
#### Предсказание по popular_recommendation 15% точности
#### Предсказание по own_purchases                  17% точности

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [11]:
# your_code